In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
VAL_IMAGE_1 = "./conditioning_image_1.png"
VAL_IMAGE_2 = "./conditioning_image_1.png" 

VAL_PROMPT_1 = "red circle with blue background"
VAL_PROMPT_2 = "cyan circle with brown floral background"

OUTPUT_DIR="controlnet-lambda"

 
DATASET_NAME = "fusing/fill50k"
PRETRAINED_MODEL = "stable-diffusion-v1-5/stable-diffusion-v1-5"

 

In [ ]:
!pip install -q accelerate>=0.16.0 torchvision datasets diffusers bitsandbytes fsspec==2025.3.2

In [ ]:
!accelerate config default

In [ ]:
!git clone https://github.com/docty/diffuser-training.git

In [ ]:
#!git clone https://huggingface.co/Docty/{MODEL_ID}

In [ ]:
wget https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/controlnet_training/conditioning_image_1.png

wget https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/controlnet_training/conditioning_image_2.png

In [ ]:
#--enable_xformers_memory_efficient_attention \
# --set_grads_to_none

In [ ]:
!accelerate launch --mixed_precision="fp16" --multi_gpu {os.getcwd()}/diffuser-training/train_control.py \
  --pretrained_model_name_or_path="$PRETRAINED_MODEL"  \
  --dataset_name="$DATASET_NAME" \
  --output_dir="$OUTPUT_DIR" \
  --resolution=128 \
  --learning_rate=1e-5 \
  --validation_image "$VAL_IMAGE_1" "$VAL_IMAGE_1" \
  --validation_prompt "$VAL_PROMPT_1" "$VAL_PROMPT_2" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --use_8bit_adam \
  --mixed_precision="fp16" \
  --tracker_project_name="controlnet-demo" \
  --push_to_hub

# Inference

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import torch


MODEL_ID = OUTPUT_DIR
controlnet_model_id = f"Docty/{MODEL_ID}"
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]


controlnet = ControlNetModel.from_pretrained(controlnet_model_id, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model_id, controlnet=controlnet, torch_dtype=torch.float16
)

# speed up diffusion process with faster scheduler and memory optimization
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
# remove following line if xformers is not installed or when using Torch 2.0.
#pipe.enable_xformers_memory_efficient_attention()
# memory optimization.
pipe.enable_model_cpu_offload()



In [ ]:
control_image = load_image(VAL_IMAGE_1)
prompt = "pale golden rod circle with old lace background"

 
generator = torch.manual_seed(0)
image = pipe(
    prompt, num_inference_steps=20, generator=generator, image=control_image
).images[0]

image